In [1]:
import pandas as pd
import numpy as np
import re
from openpyxl import load_workbook
from datetime import datetime

In [2]:
def names_to_months(x):
    a = re.sub("\D", "", x)[:4]
    if len(a) > 0:
        a = a[:4]
    elif x == "*":
        a = 'install'
    else:
        a = x
    return a

def remove_bottom(df):
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            if j < i:
                df.iat[i,j] = np.nan
    return df

def fill_diag(m, v):
    for i in range(m.shape[0]):
        for j in range(m.shape[1]):
            if j == i:
                m.iat[i,j] = v[i]
    return m

def change_matrix_orientation(m):
    m = m.transpose()
    m = m[m.columns[::-1]]
    n = m.shape[0]
    for i in range(n):
        for j in range(i+1):
            if j != j+n-1-i:
                m.iat[i,j] = m.iat[i,j+n-1-i]
                m.iat[i,j+n-1-i] = np.nan
    m.columns = range(n)
    return m

In [3]:
def culc_mau(df_mau, months):
    m_mau_reg = df_mau.pivot_table(index='created per month', columns='active month', values='users', aggfunc=sum).fillna(0).applymap(float)
    m_mau = m_mau_reg.drop(['install'], axis = 1)
    m_mau = remove_bottom(m_mau)
    m_mau = fill_diag(m_mau, m_mau_reg['install'])
    m_mau.index = m_mau.index.map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d'))
    m_mau = pd.concat([m_mau, pd.DataFrame(index = months)], axis = 1)
    return m_mau

def culc_regs(m_mau):
    m_regs = pd.DataFrame(np.diagonal(m_mau), columns=['installs'])
    m_regs.index = months
    return m_regs

def culc_payers(df_payers, months):
    m_payers_reg = df_payers.pivot_table(index='created per month', columns='active month', values='users', aggfunc=sum).fillna(0).applymap(float)
    m_payers = m_payers_reg.drop(['install'], axis = 1)
    m_payers = remove_bottom(m_payers)
    m_payers.index = m_payers.index.map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d'))
    m_payers = pd.concat([m_payers, pd.DataFrame(index = months)], axis = 1)
    return m_payers

def culc_revenue(df_revenue, months):
    df = df_revenue.groupby(['created per month']).sum()
    m_revenue = remove_bottom(df)
    m_revenue.index = m_revenue.index.map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d'))
    m_revenue = pd.concat([m_revenue, pd.DataFrame(index = months)], axis = 1)
    return m_revenue

In [4]:
# mau with registrations
# notref https://portraits.core.tf/app/kibana#/visualize/create?type=histogram&indexPattern=portraits&_g=(refreshInterval:(display:Off,pause:!f,value:0),time:(from:now-15m,mode:quick,to:now))&_a=(filters:!(),linked:!f,query:(query_string:(analyze_wildcard:!t,query:'created:%5B%222015-01-01T00:00:00%22%20TO%202018-03-31T23:59:59%5D%20AND%20referral:0')),uiState:(spy:(mode:(fill:!f,name:table))),vis:(aggs:!((id:'1',params:(),schema:metric,type:count),(id:'5',params:(customInterval:'2h',extended_bounds:(),field:created,interval:M,min_doc_count:1),schema:group,type:date_histogram),(id:'2',params:(filters:!(('$$hashKey':'object:2113',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topfacemobappandroid'))),label:''),('$$hashKey':'object:2123',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topfacemobappios')))),('$$hashKey':'object:2128',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:standalonetf')))),('$$hashKey':'object:2133',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:vk')))),('$$hashKey':'object:2138',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:ok')))),('$$hashKey':'object:2143',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:fb')))))),schema:segment,type:filters),(id:'4',params:(filters:!(('$$hashKey':'object:8412',input:(query:(query_string:(analyze_wildcard:!t,query:'*'))),label:''),('$$hashKey':'object:8421',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1501.any')))),('$$hashKey':'object:8425',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1502.any')))),('$$hashKey':'object:8431',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1503.any')))),('$$hashKey':'object:4826',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1504.any')))),('$$hashKey':'object:4839',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1505.any')))),('$$hashKey':'object:4844',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1506.any')))),('$$hashKey':'object:4849',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1507.any')))),('$$hashKey':'object:4854',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1508.any')))),('$$hashKey':'object:4859',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1509.any')))),('$$hashKey':'object:4864',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1510.any')))),('$$hashKey':'object:4869',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1511.any')))),('$$hashKey':'object:4874',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1512.any')))),('$$hashKey':'object:4879',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1601.any')))),('$$hashKey':'object:4884',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1602.any')))),('$$hashKey':'object:4889',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1603.any')))),('$$hashKey':'object:4894',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1604.any')))),('$$hashKey':'object:4899',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1605.any')))),('$$hashKey':'object:4904',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1606.any')))),('$$hashKey':'object:4909',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1607.any')))),('$$hashKey':'object:4922',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1608.any')))),('$$hashKey':'object:4927',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1609.any')))),('$$hashKey':'object:4932',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1610.any')))),('$$hashKey':'object:4937',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1611.any')))),('$$hashKey':'object:4942',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1612.any')))),('$$hashKey':'object:4947',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1701.any')))),('$$hashKey':'object:4952',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1702.any')))),('$$hashKey':'object:4957',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1703.any')))),('$$hashKey':'object:4962',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1704.any')))),('$$hashKey':'object:4967',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1705.any')))),('$$hashKey':'object:4972',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1706.any')))),('$$hashKey':'object:4985',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1707.any')))),('$$hashKey':'object:4990',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1708.any')))),('$$hashKey':'object:4995',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1709.any')))),('$$hashKey':'object:5000',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1710.any')))),('$$hashKey':'object:5005',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1711.any')))),('$$hashKey':'object:3579',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1712.any')))),('$$hashKey':'object:3584',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1801.any')))),('$$hashKey':'object:3589',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1802.any')))),('$$hashKey':'object:893',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1803.any'))))),row:!t),schema:split,type:filters)),listeners:(),params:(addLegend:!t,addTimeMarker:!f,addTooltip:!t,defaultYExtents:!f,mode:stacked,scale:linear,setYExtents:!f,shareYAxis:!t,times:!(),yAxis:()),title:'New%20Visualization',type:histogram))
# ref    https://portraits.core.tf/app/kibana#/visualize/create?type=histogram&indexPattern=portraits&_g=(refreshInterval:(display:Off,pause:!f,value:0),time:(from:now-15m,mode:quick,to:now))&_a=(filters:!(),linked:!f,query:(query_string:(analyze_wildcard:!t,query:'created:%5B%222015-01-01T00:00:00%22%20TO%202018-03-31T23:59:59%5D%20AND%20referral:1')),uiState:(spy:(mode:(fill:!f,name:table))),vis:(aggs:!((id:'1',params:(),schema:metric,type:count),(id:'5',params:(customInterval:'2h',extended_bounds:(),field:created,interval:M,min_doc_count:1),schema:group,type:date_histogram),(id:'2',params:(filters:!(('$$hashKey':'object:2113',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topfacemobappandroid'))),label:''),('$$hashKey':'object:2123',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topfacemobappios')))),('$$hashKey':'object:2128',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:standalonetf')))),('$$hashKey':'object:2133',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:vk')))),('$$hashKey':'object:2138',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:ok')))),('$$hashKey':'object:2143',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:fb')))))),schema:segment,type:filters),(id:'4',params:(filters:!(('$$hashKey':'object:8412',input:(query:(query_string:(analyze_wildcard:!t,query:'*'))),label:''),('$$hashKey':'object:8421',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1501.any')))),('$$hashKey':'object:8425',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1502.any')))),('$$hashKey':'object:8431',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1503.any')))),('$$hashKey':'object:4826',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1504.any')))),('$$hashKey':'object:4839',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1505.any')))),('$$hashKey':'object:4844',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1506.any')))),('$$hashKey':'object:4849',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1507.any')))),('$$hashKey':'object:4854',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1508.any')))),('$$hashKey':'object:4859',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1509.any')))),('$$hashKey':'object:4864',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1510.any')))),('$$hashKey':'object:4869',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1511.any')))),('$$hashKey':'object:4874',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1512.any')))),('$$hashKey':'object:4879',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1601.any')))),('$$hashKey':'object:4884',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1602.any')))),('$$hashKey':'object:4889',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1603.any')))),('$$hashKey':'object:4894',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1604.any')))),('$$hashKey':'object:4899',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1605.any')))),('$$hashKey':'object:4904',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1606.any')))),('$$hashKey':'object:4909',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1607.any')))),('$$hashKey':'object:4922',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1608.any')))),('$$hashKey':'object:4927',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1609.any')))),('$$hashKey':'object:4932',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1610.any')))),('$$hashKey':'object:4937',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1611.any')))),('$$hashKey':'object:4942',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1612.any')))),('$$hashKey':'object:4947',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1701.any')))),('$$hashKey':'object:4952',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1702.any')))),('$$hashKey':'object:4957',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1703.any')))),('$$hashKey':'object:4962',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1704.any')))),('$$hashKey':'object:4967',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1705.any')))),('$$hashKey':'object:4972',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1706.any')))),('$$hashKey':'object:4985',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1707.any')))),('$$hashKey':'object:4990',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1708.any')))),('$$hashKey':'object:4995',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1709.any')))),('$$hashKey':'object:5000',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1710.any')))),('$$hashKey':'object:5005',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1711.any')))),('$$hashKey':'object:3579',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1712.any')))),('$$hashKey':'object:3584',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1801.any')))),('$$hashKey':'object:3589',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1802.any')))),('$$hashKey':'object:893',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:user_mau.1803.any'))))),row:!t),schema:split,type:filters)),listeners:(),params:(addLegend:!t,addTimeMarker:!f,addTooltip:!t,defaultYExtents:!f,mode:stacked,scale:linear,setYExtents:!f,shareYAxis:!t,times:!(),yAxis:()),title:'New%20Visualization',type:histogram))

# paying part
# notref https://portraits.core.tf/app/kibana#/visualize/create?type=histogram&indexPattern=portraits&_g=(refreshInterval:(display:Off,pause:!f,value:0),time:(from:now-15m,mode:quick,to:now))&_a=(filters:!(),linked:!f,query:(query_string:(analyze_wildcard:!t,query:'created:%5B%222015-01-01T00:00:00%22%20TO%202018-03-31T23:59:59%5D%20AND%20referral:0')),uiState:(spy:(mode:(fill:!f,name:!n))),vis:(aggs:!((id:'1',params:(),schema:metric,type:count),(id:'5',params:(customInterval:'2h',extended_bounds:(),field:created,interval:M,min_doc_count:1),schema:group,type:date_histogram),(id:'2',params:(filters:!(('$$hashKey':'object:2113',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topfacemobappandroid'))),label:''),('$$hashKey':'object:2123',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topfacemobappios')))),('$$hashKey':'object:2128',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:standalonetf')))),('$$hashKey':'object:2133',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:vk')))),('$$hashKey':'object:2138',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:ok')))),('$$hashKey':'object:2143',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:fb')))))),schema:segment,type:filters),(id:'4',params:(filters:!(('$$hashKey':'object:8412',input:(query:(query_string:(analyze_wildcard:!t,query:'*'))),label:''),('$$hashKey':'object:8421',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1501.total')))),('$$hashKey':'object:8425',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1502.total')))),('$$hashKey':'object:8431',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1503.total')))),('$$hashKey':'object:4826',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1504.total')))),('$$hashKey':'object:4839',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1505.total')))),('$$hashKey':'object:4844',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1506.total')))),('$$hashKey':'object:4849',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1507.total')))),('$$hashKey':'object:4854',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1508.total')))),('$$hashKey':'object:4859',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1509.total')))),('$$hashKey':'object:4864',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1510.total')))),('$$hashKey':'object:4869',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1511.total')))),('$$hashKey':'object:4874',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1512.total')))),('$$hashKey':'object:4879',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1601.total')))),('$$hashKey':'object:4884',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1602.total')))),('$$hashKey':'object:4889',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1603.total')))),('$$hashKey':'object:4894',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1604.total')))),('$$hashKey':'object:4899',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1605.total')))),('$$hashKey':'object:4904',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1606.total')))),('$$hashKey':'object:4909',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1607.total')))),('$$hashKey':'object:4922',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1608.total')))),('$$hashKey':'object:4927',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1609.total')))),('$$hashKey':'object:4932',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1610.total')))),('$$hashKey':'object:4937',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1611.total')))),('$$hashKey':'object:4942',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1612.total')))),('$$hashKey':'object:4947',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1701.total')))),('$$hashKey':'object:4952',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1702.total')))),('$$hashKey':'object:4957',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1703.total')))),('$$hashKey':'object:4962',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1704.total')))),('$$hashKey':'object:4967',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1705.total')))),('$$hashKey':'object:4972',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1706.total')))),('$$hashKey':'object:4985',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1707.total')))),('$$hashKey':'object:4990',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1708.total')))),('$$hashKey':'object:4995',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1709.total')))),('$$hashKey':'object:5000',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1710.total')))),('$$hashKey':'object:5005',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1711.total')))),('$$hashKey':'object:3579',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1712.total')))),('$$hashKey':'object:3584',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1801.total')))),('$$hashKey':'object:3589',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1802.total')))),('$$hashKey':'object:1330',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1803.total'))))),row:!t),schema:split,type:filters)),listeners:(),params:(addLegend:!t,addTimeMarker:!f,addTooltip:!t,defaultYExtents:!f,mode:stacked,scale:linear,setYExtents:!f,shareYAxis:!t,times:!(),yAxis:()),title:'New%20Visualization',type:histogram))
# ref    https://portraits.core.tf/app/kibana#/visualize/create?type=histogram&indexPattern=portraits&_g=(refreshInterval:(display:Off,pause:!f,value:0),time:(from:now-15m,mode:quick,to:now))&_a=(filters:!(),linked:!f,query:(query_string:(analyze_wildcard:!t,query:'created:%5B%222015-01-01T00:00:00%22%20TO%202018-03-31T23:59:59%5D%20AND%20referral:1')),uiState:(spy:(mode:(fill:!f,name:table))),vis:(aggs:!((id:'1',params:(),schema:metric,type:count),(id:'5',params:(customInterval:'2h',extended_bounds:(),field:created,interval:M,min_doc_count:1),schema:group,type:date_histogram),(id:'2',params:(filters:!(('$$hashKey':'object:2113',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topfacemobappandroid'))),label:''),('$$hashKey':'object:2123',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topfacemobappios')))),('$$hashKey':'object:2128',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:standalonetf')))),('$$hashKey':'object:2133',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:vk')))),('$$hashKey':'object:2138',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:ok')))),('$$hashKey':'object:2143',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:fb')))))),schema:segment,type:filters),(id:'4',params:(filters:!(('$$hashKey':'object:8412',input:(query:(query_string:(analyze_wildcard:!t,query:'*'))),label:''),('$$hashKey':'object:8421',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1501.total')))),('$$hashKey':'object:8425',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1502.total')))),('$$hashKey':'object:8431',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1503.total')))),('$$hashKey':'object:4826',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1504.total')))),('$$hashKey':'object:4839',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1505.total')))),('$$hashKey':'object:4844',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1506.total')))),('$$hashKey':'object:4849',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1507.total')))),('$$hashKey':'object:4854',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1508.total')))),('$$hashKey':'object:4859',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1509.total')))),('$$hashKey':'object:4864',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1510.total')))),('$$hashKey':'object:4869',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1511.total')))),('$$hashKey':'object:4874',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1512.total')))),('$$hashKey':'object:4879',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1601.total')))),('$$hashKey':'object:4884',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1602.total')))),('$$hashKey':'object:4889',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1603.total')))),('$$hashKey':'object:4894',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1604.total')))),('$$hashKey':'object:4899',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1605.total')))),('$$hashKey':'object:4904',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1606.total')))),('$$hashKey':'object:4909',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1607.total')))),('$$hashKey':'object:4922',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1608.total')))),('$$hashKey':'object:4927',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1609.total')))),('$$hashKey':'object:4932',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1610.total')))),('$$hashKey':'object:4937',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1611.total')))),('$$hashKey':'object:4942',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1612.total')))),('$$hashKey':'object:4947',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1701.total')))),('$$hashKey':'object:4952',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1702.total')))),('$$hashKey':'object:4957',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1703.total')))),('$$hashKey':'object:4962',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1704.total')))),('$$hashKey':'object:4967',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1705.total')))),('$$hashKey':'object:4972',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1706.total')))),('$$hashKey':'object:4985',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1707.total')))),('$$hashKey':'object:4990',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1708.total')))),('$$hashKey':'object:4995',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1709.total')))),('$$hashKey':'object:5000',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1710.total')))),('$$hashKey':'object:5005',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1711.total')))),('$$hashKey':'object:3579',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1712.total')))),('$$hashKey':'object:3584',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1801.total')))),('$$hashKey':'object:3589',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1802.total')))),('$$hashKey':'object:1330',input:(query:(query_string:(analyze_wildcard:!t,query:'_exists_:revenue_in_month.1803.total'))))),row:!t),schema:split,type:filters)),listeners:(),params:(addLegend:!t,addTimeMarker:!f,addTooltip:!t,defaultYExtents:!f,mode:stacked,scale:linear,setYExtents:!f,shareYAxis:!t,times:!(),yAxis:()),title:'New%20Visualization',type:histogram))

# revenue
# revenue 1501 - 1607 total		https://portraits.core.tf/app/kibana#/visualize/create?type=histogram&indexPattern=portraits&_g=(refreshInterval:(display:Off,pause:!f,value:0),time:(from:now-15m,mode:quick,to:now))&_a=(filters:!(),linked:!f,query:(query_string:(analyze_wildcard:!t,query:'created:%5B%222015-01-01T00:00:00%22%20TO%20%222018-03-31T23:59:59%22%5D')),uiState:(spy:(mode:(fill:!f,name:table))),vis:(aggs:!((id:'1',params:(field:revenue_in_month.1501.total),schema:metric,type:sum),(id:'2',params:(field:revenue_in_month.1502.total),schema:metric,type:sum),(id:'3',params:(field:revenue_in_month.1503.total),schema:metric,type:sum),(id:'4',params:(field:revenue_in_month.1504.total),schema:metric,type:sum),(id:'5',params:(field:revenue_in_month.1505.total),schema:metric,type:sum),(id:'6',params:(field:revenue_in_month.1506.total),schema:metric,type:sum),(id:'21',params:(filters:!(('$$hashKey':'object:3820',input:(query:(query_string:(analyze_wildcard:!t,query:'referral:0'))),label:''),('$$hashKey':'object:3829',input:(query:(query_string:(analyze_wildcard:!t,query:'referral:1'))))),row:!t),schema:split,type:filters),(id:'20',params:(filters:!(('$$hashKey':'object:3756',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topfacemobappandroid'))),label:''),('$$hashKey':'object:3765',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topfacemobappios')))),('$$hashKey':'object:3770',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:standalonetf')))),('$$hashKey':'object:3775',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:vk')))),('$$hashKey':'object:3780',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:ok')))),('$$hashKey':'object:3785',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:fb')))))),schema:group,type:filters),(id:'19',params:(customInterval:'2h',extended_bounds:(),field:created,interval:M,min_doc_count:1),schema:segment,type:date_histogram),(id:'22',params:(field:revenue_in_month.1507.total),schema:metric,type:sum),(id:'23',params:(field:revenue_in_month.1508.total),schema:metric,type:sum),(id:'24',params:(field:revenue_in_month.1509.total),schema:metric,type:sum),(id:'25',params:(field:revenue_in_month.1510.total),schema:metric,type:sum),(id:'26',params:(field:revenue_in_month.1511.total),schema:metric,type:sum),(id:'27',params:(field:revenue_in_month.1512.total),schema:metric,type:sum),(id:'28',params:(field:revenue_in_month.1601.total),schema:metric,type:sum),(id:'29',params:(field:revenue_in_month.1602.total),schema:metric,type:sum),(id:'30',params:(field:revenue_in_month.1603.total),schema:metric,type:sum),(id:'31',params:(field:revenue_in_month.1604.total),schema:metric,type:sum),(id:'32',params:(field:revenue_in_month.1605.total),schema:metric,type:sum),(id:'33',params:(field:revenue_in_month.1606.total),schema:metric,type:sum),(id:'34',params:(field:revenue_in_month.1607.total),schema:metric,type:sum)),listeners:(),params:(addLegend:!t,addTimeMarker:!f,addTooltip:!t,defaultYExtents:!f,mode:stacked,scale:linear,setYExtents:!f,shareYAxis:!t,times:!(),yAxis:()),title:'New%20Visualization',type:histogram))
# revenue 1608 - 1802 total		https://portraits.core.tf/app/kibana#/visualize/create?type=histogram&indexPattern=portraits&_g=(refreshInterval:(display:Off,pause:!f,value:0),time:(from:now-15m,mode:quick,to:now))&_a=(filters:!(),linked:!f,query:(query_string:(analyze_wildcard:!t,query:'created:%5B%222015-01-01T00:00:00%22%20TO%20%222018-03-31T23:59:59%22%5D')),uiState:(spy:(mode:(fill:!f,name:table))),vis:(aggs:!((id:'21',params:(filters:!(('$$hashKey':'object:3820',input:(query:(query_string:(analyze_wildcard:!t,query:'referral:0'))),label:''),('$$hashKey':'object:3829',input:(query:(query_string:(analyze_wildcard:!t,query:'referral:1'))))),row:!t),schema:split,type:filters),(id:'20',params:(filters:!(('$$hashKey':'object:3756',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topfacemobappandroid'))),label:''),('$$hashKey':'object:3765',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topfacemobappios')))),('$$hashKey':'object:3770',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:standalonetf')))),('$$hashKey':'object:3775',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:vk')))),('$$hashKey':'object:3780',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:ok')))),('$$hashKey':'object:3785',input:(query:(query_string:(analyze_wildcard:!t,query:'first_app:topface%20AND%20platform:fb')))))),schema:group,type:filters),(id:'19',params:(customInterval:'2h',extended_bounds:(),field:created,interval:M,min_doc_count:1),schema:segment,type:date_histogram),(id:'35',params:(field:revenue_in_month.1608.total),schema:metric,type:sum),(id:'36',params:(field:revenue_in_month.1609.total),schema:metric,type:sum),(id:'37',params:(field:revenue_in_month.1610.total),schema:metric,type:sum),(id:'38',params:(field:revenue_in_month.1611.total),schema:metric,type:sum),(id:'39',params:(field:revenue_in_month.1612.total),schema:metric,type:sum),(id:'40',params:(field:revenue_in_month.1701.total),schema:metric,type:sum),(id:'41',params:(field:revenue_in_month.1702.total),schema:metric,type:sum),(id:'42',params:(field:revenue_in_month.1703.total),schema:metric,type:sum),(id:'43',params:(field:revenue_in_month.1704.total),schema:metric,type:sum),(id:'44',params:(field:revenue_in_month.1705.total),schema:metric,type:sum),(id:'45',params:(field:revenue_in_month.1706.total),schema:metric,type:sum),(id:'46',params:(field:revenue_in_month.1707.total),schema:metric,type:sum),(id:'47',params:(field:revenue_in_month.1708.total),schema:metric,type:sum),(id:'48',params:(field:revenue_in_month.1709.total),schema:metric,type:sum),(id:'49',params:(field:revenue_in_month.1710.total),schema:metric,type:sum),(id:'50',params:(field:revenue_in_month.1711.total),schema:metric,type:sum),(id:'51',params:(field:revenue_in_month.1712.total),schema:metric,type:sum),(id:'52',params:(field:revenue_in_month.1801.total),schema:metric,type:sum),(id:'53',params:(field:revenue_in_month.1802.total),schema:metric,type:sum),(id:'54',params:(field:revenue_in_month.1803.total),schema:metric,type:sum)),listeners:(),params:(addLegend:!t,addTimeMarker:!f,addTooltip:!t,defaultYExtents:!f,mode:stacked,scale:linear,setYExtents:!f,shareYAxis:!t,times:!(),yAxis:()),title:'New%20Visualization',type:histogram))

In [5]:
stop_datetime = '2018-04-01 00:00:00'
df_ref_mau = pd.read_csv(r'./data/ref_mau.csv',thousands=',')
df_notref_mau = pd.read_csv(r'./data/notref_mau.csv',thousands=',') 
df_ref_mau['ref'] = 'ref'
df_notref_mau['ref'] = 'notref'
df_mau = df_notref_mau.append(df_ref_mau)
df_mau.columns = ['created per month', 'app', 'active month', 'users', 'ref']
df_mau = df_mau[df_mau['created per month'] < stop_datetime]
df_mau['active month'] = df_mau['active month'].map(lambda x: names_to_months(x))

In [6]:
df_revenue = pd.read_csv(r'./data/revenue_1.csv',thousands=',').append(pd.read_csv(r'./data/revenue_2.csv',thousands=','))
df_revenue.rename(columns={'filters': 'ref', 'filters.1': 'app'}, inplace=True)
df_revenue = df_revenue[df_revenue['created per month'] < stop_datetime]
df_revenue.columns = df_revenue.columns.map(lambda x: names_to_months(x))
df_revenue = df_revenue.fillna(0)

In [7]:
df_ref_payers = pd.read_csv(r'./data/ref_payers.csv',thousands=',')
df_notref_payers = pd.read_csv(r'./data/notref_payers.csv',thousands=',')
df_ref_payers['ref'] = 'ref'
df_notref_payers['ref'] = 'notref'
df_payers = df_notref_payers.append(df_ref_payers)
df_payers.columns = ['created per month', 'app', 'active month', 'users', 'ref']
df_payers = df_payers[df_payers['created per month'] < stop_datetime]
df_payers['active month'] = df_payers['active month'].map(lambda x: re.sub("\D", "", x)[:4] if len(x) > 1 else 'install')

In [8]:
dict_app = {'first_app:topfacemobappandroid': 'android','first_app:topfacemobappios':'ios','first_app:standalonetf':'st',
            'first_app:topface AND platform:vk' : 'vk', 'first_app:topface AND platform:ok': 'ok','first_app:topface AND platform:fb': 'fb'}
dict_ref = {'referral:0': 'notref','referral:1':'ref'}
df_mau['app'] = df_mau['app'].map(dict_app)
df_payers['app'] = df_payers['app'].map(dict_app)
df_revenue['app'] = df_revenue['app'].map(dict_app)
df_revenue['ref'] = df_revenue['ref'].map(dict_ref)

In [9]:
months = sorted(set(df_mau['created per month'].map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").strftime('%Y-%m-%d'))))
apps = list(set(df_mau['app']))
refs = list(set(df_mau['ref']))
variable_list = ['regs','mau','payers','revenue','retention','payers_part','arpu_monthly','arppu_monthly','ltv_equal','ltv_cumsum']

In [10]:
for ref in refs:
    for app in apps:
        m_mau = culc_mau(df_mau[(df_mau['app'] == app) & (df_mau['ref'] == ref)], months)
        m_regs = culc_regs(m_mau)
        m_payers = culc_payers(df_payers[(df_payers['app'] == app) & (df_payers['ref'] == ref)], months)
        m_revenue = culc_revenue(df_revenue[(df_revenue['app'] == app) & (df_revenue['ref'] == ref)], months)
        m_retention = m_mau.divide(m_regs['installs'], axis='index')
        m_payers_part = m_payers.divide(m_mau)
        m_arpu_monthly = m_revenue.divide(m_mau)
        m_arppu_monthly = m_revenue.divide(m_payers)
        m_ltv_equal = m_revenue.divide(m_regs['installs'], axis='index')
        m_ltv_cumsum = np.cumsum(m_ltv_equal, axis=1)
        
        writer = pd.ExcelWriter('./out_round/cohort_analysis' + '_' + ref + '_' + app + '.xlsx')
        for variable in variable_list:
            if variable == 'regs':
                round(vars()['m_' + variable], 0).to_excel(writer,variable)
            elif variable in ['mau','payers','revenue']:
                round(change_matrix_orientation(vars()['m_' + variable]), 0).to_excel(writer,variable)
            elif variable in ['retention','payers_part']:
                round(change_matrix_orientation(vars()['m_' + variable]) * 100, 2).to_excel(writer,variable)
            elif variable in ['arpu_monthly','arppu_monthly','ltv_equal','ltv_cumsum']:
                round(change_matrix_orientation(vars()['m_' + variable]), 4).to_excel(writer,variable)
        writer.save()
        
        print(ref, app, m_mau.shape[0], m_regs.shape[0], m_payers.shape[0], m_revenue.shape[0], m_retention.shape[0], m_payers_part.shape[0], m_arpu_monthly.shape[0], m_arppu_monthly.shape[0],m_ltv_equal.shape[0], m_ltv_cumsum.shape[0])
     
        #writer = pd.ExcelWriter('./out/cohort_analysis' + '_' + ref + '_' + app + '.xlsx')
        #for variable in variable_list:
        #     vars()['m_' + variable].to_excel(writer,variable)
        #writer.save()
        
        #for variable in variable_list:
        #     vars()['m_' + variable +'_' + ref + '_' + app] = vars()['m_' + variable]
        #writer.save()

notref android 39 39 39 39 39 39 39 39 39 39
notref st 39 39 39 39 39 39 39 39 39 39
notref fb 39 39 39 39 39 39 39 39 39 39
notref vk 39 39 39 39 39 39 39 39 39 39
notref ios 39 39 39 39 39 39 39 39 39 39
notref ok 39 39 39 39 39 39 39 39 39 39
ref android 39 39 39 39 39 39 39 39 39 39
ref st 39 39 39 39 39 39 39 39 39 39
ref fb 39 39 39 39 39 39 39 39 39 39
ref vk 39 39 39 39 39 39 39 39 39 39
ref ios 39 39 39 39 39 39 39 39 39 39
ref ok 39 39 39 39 39 39 39 39 39 39
